# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models_tf2 import Models
from botbidder import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'), False)  # Load sampling strategies


Loading config file d:\GitHub\ben\src\./config/default.conf
Loading config file d:\GitHub\ben\src\./config/default.conf


### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# you sit West and hold:
hand = 'KJ7432.9.A.QJT86'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "PAD_START", "PASS", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 2, dds, False, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '1S',
  'insta_score': 1.172,
  'alert': 'False',
  'explanation': '5+ !S -- 6-!H; 5+!S; 11-20 HCP'}]

In [3]:
auction = ["PAD_START", "PAD_START", "PASS", "PASS", "1S", "2H", "PASS", "3D"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 1.082,
  'adjustment': 0,
  'expected_score': -75,
  'expected_tricks': 9.6,
  'alert': 'False',
  'explanation': ' -- 6-!H; 5+!S; 11-20 HCP'},
 {'call': '3S', 'insta_score': 0.115}]

In [4]:
bid.samples

['KJxxxx.x.A.QJTxx x.AKJxxx.Kxx.Axx x.QTxxx.xxx.Kxxx AQTxx.x.QJTxxx.x - 0.84457',
 'KJxxxx.x.A.QJTxx AQ.KQJTxx.x.Axxx x.xxxx.Txxxx.Kxx Txxx.Ax.KQJxxx.x - 0.80495',
 'KJxxxx.x.A.QJTxx Qxx.AQTxxx.Tx.Ax .KJxxx.Kxxx.xxxx ATxx.x.QJxxxx.Kx - 0.80252',
 'KJxxxx.x.A.QJTxx x.AKJxxx.KTxx.Kx Tx.Txxxx.xxx.Axx AQxx.Q.QJxxx.xxx - 0.79120',
 'KJxxxx.x.A.QJTxx x.QJTxxx.Qx.AKxx Tx.AKxx.xxxx.xxx AQxx.xx.KJTxxx.x - 0.79102',
 'KJxxxx.x.A.QJTxx Qx.AKxxxx.Jxx.Kx x.QJxx.xxx.Axxxx ATxx.Tx.KQTxxx.x - 0.78081',
 'KJxxxx.x.A.QJTxx x.AKQTxxxx.Kxx.x xx.xxx.xxx.AKxxx AQTx.J.QJTxxx.xx - 0.78075',
 'KJxxxx.x.A.QJTxx QTx.KJxxx.Q.AKxx .ATxxx.Jxxxx.xxx Axxx.Qx.KTxxxx.x - 0.77850',
 'KJxxxx.x.A.QJTxx ATx.AKQxxx.Jxx.x x.xxxx.Txx.Axxxx Qxx.JT.KQxxxx.Kx - 0.77810',
 'KJxxxx.x.A.QJTxx Qx.AKTxx.xxx.Kxx T.Qxxxx.Jxx.Axxx Axxx.Jx.KQTxxx.x - 0.77784',
 'KJxxxx.x.A.QJTxx xx.KQJTxx.x.AKxx Q.xxxx.KJxxx.xxx ATxx.Ax.QTxxxx.x - 0.77382',
 'KJxxxx.x.A.QJTxx x.AKQJTxxx.xxx.K A.xxx.Qxxx.xxxxx QTxxx.x.KJTxx.Ax - 0.76714',
 'KJxxxx.x.A.QJT

In [5]:
auction = ["PAD_START", "PAD_START", "PASS", "PASS", "1S", "2H", "PASS", "3D", "PASS", "3N" , "PASS", "PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 1.199,
  'adjustment': 0,
  'expected_score': -288,
  'expected_tricks': 8.9,
  'alert': 'False',
  'explanation': ' -- 6-!H; 5+!S; 11-20 HCP'}]

In [7]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 1, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 2, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 3, :], models.n_cards_bidding)}')

Good quality: 0.8848420595606045
KJxxxx.x.A.QJTxx AQx.KTxx.xxxxx.x .Jxxx.KQTx.Axxxx Txxx.AQxx.Jxx.Kx
KJxxxx.x.A.QJTxx AQT.Jxxx.JTx.xxx x.Kxxx.KQxx.Axxx xxx.AQTx.xxxxx.K
KJxxxx.x.A.QJTxx Axxx.Txx.Qxxx.Ax .AQxx.JTxxx.Kxxx QTx.KJxxx.Kxx.xx
KJxxxx.x.A.QJTxx AQxx.Qxx.KJTxx.x T.JTxx.Qxxx.AKxx xx.AKxxx.xxx.xxx
KJxxxx.x.A.QJTxx AQx.KTxx.Qxxxx.x x.AQJx.Jxxx.Kxxx Txx.xxxx.KTx.Axx
KJxxxx.x.A.QJTxx Axx.Kxxx.KJxx.xx T.AQxx.Qxxx.Axxx Qxx.JTxx.Txxx.Kx
KJxxxx.x.A.QJTxx Txx.Kxx.QTxx.Axx x.Axxx.KJxx.Kxxx AQx.QJTxx.xxxx.x
KJxxxx.x.A.QJTxx AQT.AJxx.Qxx.xxx .KTxx.KJxxx.Kxxx xxxx.Qxxx.Txxx.A
KJxxxx.x.A.QJTxx Qxxx.ATxx.QJxx.A .KJxx.Kxxx.Kxxxx ATx.Qxxx.Txxx.xx
KJxxxx.x.A.QJTxx Qxxx.KQxx.JTxx.x x.Axxx.Qxxx.AKxx AT.JTxx.Kxxx.xxx
KJxxxx.x.A.QJTxx Axx.AQx.Txxxx.xx x.KTxx.KQJx.Axxx QTx.Jxxxx.xxx.Kx
KJxxxx.x.A.QJTxx Axxx.ATxx.xx.Kxx .Qxxx.KQJx.Axxxx QTx.KJxx.Txxxxx.
KJxxxx.x.A.QJTxx Qx.KQJxx.xxxx.xx x.ATxx.KTxx.Axxx ATxx.xxx.QJxx.Kx
KJxxxx.x.A.QJTxx Axx.Jxxx.QTxx.Ax .ATxx.KJxxx.Kxxx QTxx.KQxx.xxx.xx
KJxxxx.x.A.QJTx